In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn import * 

In [4]:
a = tf.constant(1.0)
a

<tf.Tensor 'Const:0' shape=() dtype=float32>

In [5]:
b = tf.constant(1.0)
b

<tf.Tensor 'Const_1:0' shape=() dtype=float32>

In [8]:
c = a + b
c

<tf.Tensor 'add_2:0' shape=() dtype=float32>

In [10]:
with tf.Session() as sess:
    c_ = sess.run(c)
    print(c_)

2.0


In [17]:
df = pd.read_csv("/data/bank-full.csv", sep=";")
y = np.where(df.y == "yes", 1, 0)
X = df.iloc[:, 0:-1]
X = pd.get_dummies(X, drop_first=True)

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = 0.3, random_state = 1)
scaler = preprocessing.StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [19]:
pd.DataFrame(X_train).head()

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,-0.841435,-0.513906,1.347581,0.654165,-0.563213,-0.410057,-0.299354,1.900682,-0.186341,-0.166834,...,-0.426206,-0.364749,-0.101863,1.511034,-0.311612,-0.129251,-0.115553,-0.204721,-0.18534,0.471004
1,-0.652995,-0.590185,1.347581,-0.572683,0.391184,-0.410057,-0.299354,-0.526127,-0.186341,-0.166834,...,-0.426206,2.741608,-0.101863,-0.661798,-0.311612,-0.129251,-0.115553,-0.204721,-0.18534,0.471004
2,-0.841435,-0.353719,0.385408,-0.351385,-0.245081,-0.410057,-0.299354,-0.526127,-0.186341,-0.166834,...,-0.426206,-0.364749,-0.101863,-0.661798,-0.311612,-0.129251,-0.115553,-0.204721,-0.18534,0.471004
3,-0.464554,-0.434130,-1.538940,-0.421268,0.391184,-0.410057,-0.299354,1.900682,-0.186341,-0.166834,...,-0.426206,-0.364749,-0.101863,-0.661798,-0.311612,-0.129251,-0.115553,-0.204721,-0.18534,0.471004
4,-1.689418,-0.283797,-1.538940,-0.665862,-0.563213,-0.410057,-0.299354,-0.526127,-0.186341,-0.166834,...,-0.426206,-0.364749,9.817097,-0.661798,-0.311612,-0.129251,-0.115553,-0.204721,-0.18534,0.471004


In [44]:
n_x = X_train.shape[1]
n_y = 2
learning_rate = 0.01
epochs = 100

X = tf.placeholder(dtype=tf.float32, shape=[None, n_x])
y = tf.placeholder(dtype=tf.int32, shape=[None])

print("X", X)
print("y", y)

W = tf.Variable(tf.truncated_normal(shape=[n_x, n_y], stddev=0.1))
print("W", W)

b = tf.Variable(tf.zeros(shape=[n_y]))
print("b", b)


y_oh = tf.one_hot(y, depth=n_y)
print("y_oh: ", y_oh)

Z = tf.matmul(X, W) + b
print("Z", Z)

A = tf.nn.softmax(Z)
print("Activation:", A)

y_pred = tf.argmax(Z, axis = 1, output_type=tf.int32)
print("y_pred", y_pred)

accuracy = tf.reduce_mean(tf.cast(tf.equal(y, y_pred), tf.float32))
print("Accuracy", accuracy)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_oh, logits=Z))

print("cost: ", cost)

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(epochs):
        _, cost_, accuracy_ = sess.run([optimizer, cost, accuracy]
                                    , feed_dict={X: X_train, y: y_train})
        if i % (epochs // 20) == 0:
            print(accuracy_)
    test_accuracy_, y_pred_ = sess.run([accuracy, y_pred], feed_dict={X: X_test, y: y_test}) 
    print("Test accuracy: ", test_accuracy_)



X Tensor("Placeholder_48:0", shape=(?, 42), dtype=float32)
y Tensor("Placeholder_49:0", shape=(?,), dtype=int32)
W <tf.Variable 'Variable_47:0' shape=(42, 2) dtype=float32_ref>
b <tf.Variable 'Variable_48:0' shape=(2,) dtype=float32_ref>
y_oh:  Tensor("one_hot_21:0", shape=(?, 2), dtype=float32)
Z Tensor("add_22:0", shape=(?, 2), dtype=float32)
Activation: Tensor("Softmax_16:0", shape=(?, 2), dtype=float32)
y_pred Tensor("ArgMax_17:0", shape=(?,), dtype=int32)
Accuracy Tensor("Mean_21:0", shape=(), dtype=float32)
cost:  Tensor("Mean_22:0", shape=(), dtype=float32)
0.51041174
0.6830031
0.7856037
0.84902203
0.8760704
0.8850444
0.8880147
0.888615
0.89120615
0.89294404
0.8943344
0.8968307
0.8981262
0.898537
0.89967453
0.89983255
0.9002433
0.90115964
0.90191805
0.9022656
Test accuracy:  0.9034208


In [45]:
metrics.accuracy_score(y_test, y_pred_)

0.903420819817163